In [217]:
import requests
import pandas as pd
from lxml import html
from bs4 import BeautifulSoup
import datetime

In [218]:
url = "https://inshorts.com/sitemap-en.xml"
data = requests.get(url)
page = requests.get(url)
tree = html.fromstring(page.content)

In [219]:
count = len(tree.xpath("//url"))
raw_urls = []

In [220]:
str(tree.xpath("//url[2]/loc/text()")).replace("['","").replace("']","")

'https://inshorts.com/en/news/pmled-panel-may-meet-on-january-24-to-select-new-cbi-chief-1547648052932'

In [221]:
for i in range(1,count+1):
    raw_urls.append(str(tree.xpath("//url["+str(i)+"]/loc/text()")).replace("['","").replace("']",""))

In [222]:
k = []
v = []
for i in range(1,count+1):
    #l = str(tree.xpath("//url["+i+"]/loc/text()")).replace("['","").replace("']","").count("-")
    #str(tree.xpath("//url[2]/loc/text()")).replace("['","").replace("']","").split("-")[:l]
    k.append(int(str(tree.xpath("//url["+str(i)+"]/loc/text()")).replace("['","").replace("']","").split("-")[-1]))
    v.append(str(tree.xpath("//url["+str(i)+"]/loc/text()")).replace("['","").replace("']",""))

In [223]:
dict_url = {k[i]:v[i] for i in range(count)}

In [224]:
#dict_url_pandas = pd.DataFrame.from_dict(dict_url, orient='index')
dict_url_pandas = pd.DataFrame(list(dict_url.items()), columns=['news_id','news_url'])

In [225]:
dict_url_pandas.head()

,news_id,news_url
0,1547638223811,https://inshorts.com/en/news/sad-criticises-co...
1,1547648052932,https://inshorts.com/en/news/pmled-panel-may-m...
2,1547650151681,https://inshorts.com/en/news/apple-spends-$150...
3,1547651408101,https://inshorts.com/en/news/mahindra-hikes-st...
4,1547714434446,https://inshorts.com/en/news/it-wasnt-against-...


In [226]:
temp = pd.read_csv('inshorts_url.csv')
news_url_set = set(temp['news_url'])

In [227]:
dict_url_pandas_set = set(dict_url_pandas['news_url'])

In [228]:
pd.DataFrame(list((dict_url_pandas_set | news_url_set)), columns=['news_url']).to_csv('inshorts_url.csv', index=False)

In [229]:
temp.news_url[2001]

'https://inshorts.com/en/news/bjp-will-win-74-lok-sabha-seats-in-uttar-pradesh-jp-nadda-1547637538963'

In [230]:
len(temp)

2002

# Download News

url = "https://inshorts.com/en/news/death-toll-due-to-cyclone-gaja-rises-to-33-in-tamil-nadu-1542475432881"
data = requests.get(url)
page = requests.get(url)
tree = html.fromstring(page.content)

def clean_xpath(xpath):
    xpath = str(xpath).replace('["','').replace('"]','')
    xpath = str(xpath).replace("['","").replace("']","")
    return xpath

headline = clean_xpath(tree.xpath("//span[@itemprop='headline']/text()"))

author = clean_xpath(tree.xpath("//span[@class='author']/text()")[1])

time = clean_xpath(tree.xpath("//span[@class='time']/text()")[1])

date = clean_xpath(tree.xpath("/html/body/div[4]/div/div[3]/div/div/div[2]/div/span[3]/text()"))
date = "-".join(date.split(",")[0].split())

body = clean_xpath(tree.xpath("/html/body/div[4]/div/div[3]/div/div/div[3]/div[1]/text()"))

news_from = clean_xpath(tree.xpath("/html/body/div[4]/div/div[3]/div/div/div[4]/div/a/text()"))

print(headline)
print(author)
print(time)
print(date)
print(body)
print(news_from)

In [267]:
def get_news(news_url):
    data = requests.get(news_url)
    page = requests.get(url)
    tree = html.fromstring(page.content)
    headline = clean_xpath(tree.xpath("//span[@itemprop='headline']/text()"))
    author = clean_xpath(tree.xpath("//span[@class='author']/text()")[1])
    time = clean_xpath(tree.xpath("//span[@class='time']/text()")[1])
    date = clean_xpath(tree.xpath("/html/body/div[4]/div/div[3]/div/div/div[2]/div/span[3]/text()"))
    date = "-".join(date.split(",")[0].split())
    body = clean_xpath(tree.xpath("/html/body/div[4]/div/div[3]/div/div/div[3]/div[1]/text()"))
    news_from_source = clean_xpath(tree.xpath("/html/body/div[4]/div/div[3]/div/div/div[4]/div/a/text()"))
    news = (url,headline,body,author,time,date,news_from_source)
    return news

In [276]:
temp = pd.read_csv('inshorts_url.csv')

In [277]:
l = []
for i in range(len(temp)):
    l.append(get_news(temp.news_url.iloc[i]))

IndexError: list index out of range

In [275]:
news_total = pd.DataFrame.from_records(l, columns=['url','headline','body','author','time','date','source'])
news_total.to_csv('news_inshorts.csv', )